In [ ]:
# Scikit-image library for image processing tools in python
!pip3 install scikit-image
# Scikit-learn library for ML in python
!pip3 install scikit-learn
# install the library h5py (needed to load and read the ground truth relevance file of assignment 02)
!pip3 install h5py
# install the library tqdm (for progress visualization)
!pip3 install tqdm
# Matplotlib/Pylab for plots and visualization in python
!pip3 install matplotlib

### Things to try:
- use of different keypoint descriptors (e.g. SIFT, SURF, and so on) 
- study the impact of different clustering techniques to learn the visual dictionary (e.g. fuzzy k-means, soft k-means, hierarchical clustering) of different sizes of the vocabulary (values of k)
- use of different distance metrics to compare the final descriptors 
- use of Machine Learning for learning to rank


## Load the query and map images

In [ ]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tqdm

# map
with open("data02/database/database.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("data02/query/query.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])


## Load the similarity matrix

In [ ]:
# loading the relevance judgements
with h5py.File("data02/london_gt.h5","r") as f:
    fovs = f["fov"][:]
    sim = f["sim"][:].astype(np.uint8)

## Clean dataset

In [ ]:
print(sim.shape)
index = []
new_sim = []
for i in range(len(sim)):
    if np.sum(sim, axis=1)[i] != 0:
        new_sim.append(sim[i])
        
sim = np.array(new_sim)
print(sim.shape)

In [ ]:
for i in range(len(sim)):
    if np.sum(sim, axis=1)[i] == 0:
        for k in range(len(sim[i])):
            if sim[i][k] == 1:
                print("Still queries left with 0 relevant images")

## Visualize some images

In [ ]:
# generate random index for the query image
query_idx = np.random.randint(0, 499) 

# select the relevant and non-relevant map images for the randomly selected query image
rel = np.where(sim[query_idx, :] == 1)
nonrel = np.where(sim[query_idx, :] == 0)


# randomly select a relevant and non-relevant image
rel_idx = rel[0][np.random.randint(0, len(rel[0]) - 1)]
nonrel_idx = nonrel[0][np.random.randint(0, len(nonrel[0]) - 1)]

plt.figure(figsize=(17,10))
plt.subplot(1,3,1)
plt.imshow(plt.imread('data02/' + q_imgs[query_idx]))
plt.axis("off")

plt.subplot(1,3,2)
plt.title('Relevant image result (from map)')
plt.imshow(plt.imread('data02/' + m_imgs[rel_idx]))
plt.axis("off")

plt.subplot(1,3,3)
plt.title('Non-relevant image result (from map)')
plt.imshow(plt.imread('data02/' + m_imgs[nonrel_idx]))
plt.axis("off")


## Bag of words

### Compute the descriptors

In [ ]:
import os
import skimage
#from skimage.feature import ORB
from skimage.feature import SIFT
from skimage.color import rgb2gray
import cv2 

# Initialize the descriptor

#descriptor_extractor = SIFT()
descriptor_extractor = cv2.SIFT_create()
#descriptor_extractor = ORB(n_keypoints=50)
# Initialize the data structure that will contain all the descriptors
descriptors = None

# Loop over map images
for img_name in m_imgs:
    img = plt.imread(os.path.join('data02/', img_name))
    #img = rgb2gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Extract descriptors
    keypoints_surf, descriptors_img = descriptor_extractor.detectAndCompute(img, None)
    
    # Accumulate the computed descriptors
    if descriptors is None:
        descriptors = descriptors_img
    else:
        descriptors = np.vstack( (descriptors, descriptors_img))
    
print(descriptors.shape)

### Save/Load the descriptors

In [ ]:
import pickle
# save descriptors (uncomment if you want to save the computed descriptors)
f = open('data02/SIFT-full-descriptors-map.bin', 'wb')
data = pickle.dump(descriptors, f)
f.close()

# load pre-computed descriptors
f = open('data02/SIFT-full-descriptors-map.bin', 'rb')
descriptors = pickle.load(f)
f.close()


### Create the clusters

In [ ]:
from tabnanny import verbose
from cv2 import sqrt
import sklearn
from sklearn.cluster import MiniBatchKMeans

# clustering
K = 100  # number of clusters (equivalent to the number of words) we want to estimate
num_initialization = 5 # Number of time the k-means algorithm will be run with different centroid seeds.

# Run the k-means clustering
kmeans = MiniBatchKMeans(n_clusters=K, random_state=0, n_init=num_initialization, verbose=1)
clusters = kmeans.fit(descriptors)  # we use the descriptors extracted from the map (training) images before
centroids = clusters.cluster_centers_


print("Shape of the centroids matrix: ", centroids.shape)
print("We computed ", centroids.shape[0], "centroids of lengh ", centroids.shape[1], " (the same of the descriptor)")


### save/load the centroids

In [ ]:
# save centroids (uncomment if you want to save the computed centroids)
f = open('data02/SIFT-full-mini-batch-centroids-map.bin', 'wb')
data = pickle.dump(centroids, f)
f.close()

# load pre-computed centroids
f = open('data02/SIFT-full-mini-batch-centroids-map.bin', 'rb')
centroids = pickle.load(f)
f.close()

### Compute the vector

In [ ]:
from tqdm import tqdm
import os
import skimage
from skimage.feature import SIFT
from skimage.color import rgb2gray
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance


# compute the bag of word vector for an image
def bag_of_words(centroids, img_descriptors):
    n_centroids = centroids.shape[0]  # number of centroids found with the KMeans clustering
    n_descriptors = img_descriptors.shape[0]  # number of descriptors extracted from the image
    
    # initialization of the bag of words (BoW) vector
    # Note that the BoW vector has length equal to the number of cluster centroids
    # The cluster centroids are indeed our visual words, and the BoW will be the histogram of these words found in the given image
    bow_vector = np.zeros(n_centroids)  
    
    for i in range(n_descriptors):
        closest_dist = 9999
        index = 0
        for j in range(n_centroids):
            #dist = np.linalg.norm(img_descriptors[i]-centroids[j])
            dist = distance.cityblock(img_descriptors[i],centroids[j])
            if(dist < closest_dist):
                closest_dist = dist
                index = j
        bow_vector[index] += 1
    return bow_vector


# Test the implementation of the BoW vector computation
img = plt.imread(os.path.join('data02/', q_imgs[0]))
#img = rgb2gray(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

keypoints_surf, query_img_descriptors = descriptor_extractor.detectAndCompute(img, None)

bow = bag_of_words(centroids, query_img_descriptors)
print("Size of the bow vector: ", bow.shape)
print("Bow vector: ", bow)


## Convert the images to BoW representation

In [ ]:
bow_map_images = None
# loop over the images in the map set
for img_name in tqdm(m_imgs):
    # load image
    img = plt.imread(os.path.join('data02/', img_name))
    #img = rgb2gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # extract the keypoints and corresponding descriptors
    _,img_descriptors = descriptor_extractor.detectAndCompute(img, None)# descriptors (the feature vectors)
    
    # compute BoW representation of the image (using the basic 'words', i.e. centroids, computed earlier)
    bow = bag_of_words(centroids, img_descriptors)
    # add the computed BoW vector to the set of map representations
    if bow_map_images is None:
        bow_map_images = bow
    else:
        bow_map_images = np.vstack( (bow_map_images, bow))

## Normalization of data

In [ ]:
from sklearn import preprocessing
orig_bow_map_images = bow_map_images

# Compute z-score statistics
scaler = preprocessing.StandardScaler().fit(bow_map_images)
# Normalize the vectors of the map collection (0 mean and 1 std)
bow_map_images = scaler.transform(bow_map_images)


## Retrieve the images

In [ ]:
from scipy.spatial import distance
# receives as input the:
#   - bag of words vectors of the map images
#   - the bag of work vector of the query image
def retrieve_images(map_bow_vectors, query_bow):
    n_map_bow_vectors = map_bow_vectors.shape[0]
    bow_distances = np.zeros(n_map_bow_vectors)
    most_similar = None  # use this to 
    
    for i in range(n_map_bow_vectors):
        #dist = np.linalg.norm(map_bow_vectors[i]-query_bow)
        dist = distance.cityblock(map_bow_vectors[i],query_bow)
        #dist = distance.cityblock(map_bow_vectors[i],query_bow)
        bow_distances[i] = dist
    
    most_similar = np.argsort(bow_distances, axis=- 1, kind=None, order=None)
    
    
    
    return most_similar


        #dist = distance(map_bow_vectors[i]-query_bow)

# Retrieve the most similar images to query image 221 (index 221-1=220)
query_idx = 220
img = plt.imread("data02/" + q_imgs[query_idx])
#img = rgb2gray(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# compute bag of words
_,query_img_descriptors = descriptor_extractor.detectAndCompute(img, None)

bow = bag_of_words(centroids, query_img_descriptors)

# Normalize the query BoW vector using the mean and variance of the map (computed earlier and saved into the scaler object)
bow = scaler.transform(bow.reshape(-1, 1).transpose())
bow = bow.transpose().reshape(-1)

# Retrieve the indices of the top-10 similar images from the map
retrieved_images = retrieve_images(bow_map_images, bow)
print('Indices of similar images retrieved: ', retrieved_images[:10])
# Indices of the relevant map images for the query: we have the relevance judgements (Ground truth)
relevant_images = np.where(sim[query_idx, :] == 1)[0]
print('Indices of relevant images (given in the GT relevance judgements): ', relevant_images)

## Evaluate performance

#### Precision at k

In [ ]:
def precision(relevant, retrieved):
    count = 0
    if len(retrieved) > 0:
        for x in relevant:
            if x in retrieved:
                count += 1
        return count / len(retrieved)
    return 0
    
def precision_at_k(relevant, retrieved, k):
    kRetrieved = retrieved[0:k]
    return precision(relevant, kRetrieved)

prec5 = precision_at_k(relevant_images, retrieved_images, 5)
prec10 = precision_at_k(relevant_images, retrieved_images, 10)

print('P@5: ', prec5)
print('P@10: ', prec10)

#### Mean average precision

In [ ]:
def average_precision(relevant, retrieved):
    avsum = 0
    for i in range(len(retrieved)):
        if retrieved[i] in relevant:
            pr = precision(relevant, retrieved[0:i+1])
            avsum += pr
    return avsum/len(relevant)

def mean_average_precision(all_relevant, all_retrieved):
    total = 0
    count = len(all_relevant)
    for i in range(len(all_relevant)):
        total += average_precision(all_relevant[i],all_retrieved[i])
    return total / count

relevant_images_MAP = []
retrieved_images_MAP = []
for i in range(len(sim)):
    relevant_images_MAP.append(np.where(sim[i, :] == 1)[0])
    
    img = plt.imread("data02/" + q_imgs[i])
    #img = rgb2gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # compute bag of words
    _,query_img_descriptors = descriptor_extractor.detectAndCompute(img, None)
    bow = bag_of_words(centroids, query_img_descriptors)

    # Normalize the query BoW vector using the mean and variance of the map (computed earlier and saved into the scaler object)
    bow = scaler.transform(bow.reshape(-1, 1).transpose())
    bow = bow.transpose().reshape(-1)

    # Retrieve the indices of the top-10 similar images from the map
    retrieved_images_MAP.append(retrieve_images(bow_map_images, bow))

MAP = mean_average_precision(relevant_images_MAP, retrieved_images_MAP)
print('The mean average precision is: ', MAP)